# SatMAE Finetuning on Google Colab

This notebook sets up and runs SatMAE finetuning on EuroSAT dataset using Google Colab's free GPU.

## 🚀 Features:
- Automatic SatMAE repository download
- Conda environment setup using original env.yml
- EuroSAT dataset integration via Google Drive
- SatMAE model finetuning with multispectral data
- GPU acceleration (T4/V100/A100)

**Runtime**: Make sure to select **GPU** runtime (Runtime → Change runtime type → Hardware accelerator → GPU)

## 1. Pull SatMAE Code from GitHub

In [ ]:
# Clone SatMAE repository from GitHub
import os

print("? Downloading SatMAE repository from GitHub...")

if not os.path.exists('SatMAE'):
    !git clone https://github.com/pvinnbru/SatMAE.git
    print("✅ SatMAE repository cloned successfully")
else:
    print("✅ SatMAE repository already exists")

# Navigate to SatMAE directory
%cd SatMAE

# List repository contents
print("\n? Repository contents:")
!ls -la

print("\n🎉 SatMAE code ready!")

PyTorch version: 2.6.0+cu124
CUDA available: True
GPU: Tesla T4
CUDA version: 12.4
GPU memory: 15.8 GB


## 2. Environment Setup

Installing conda environment using the original SatMAE env.yml file. This ensures exact package compatibility as tested by the SatMAE authors.

In [ ]:
# Install conda environment using the original env.yml
import os
import subprocess

print("? Setting up conda environment from env.yml...")

# Check if conda is available
conda_available = subprocess.run("which conda", shell=True, capture_output=True).returncode == 0

if not conda_available:
    print("📦 Installing Miniconda...")
    !wget https://repo.anaconda.com/miniconda/Miniconda3-latest-Linux-x86_64.sh -O miniconda.sh
    !bash miniconda.sh -b -p /content/miniconda
    
    # Add conda to PATH
    os.environ['PATH'] = '/content/miniconda/bin:' + os.environ['PATH']
    
    # Initialize conda
    !source /content/miniconda/bin/activate && conda init bash
    print("✅ Miniconda installed successfully")
else:
    print("✅ Conda already available")

# Install mamba for faster package resolution
print("📦 Installing mamba for faster package resolution...")
!conda install mamba -n base -c conda-forge -y

# Create conda environment from env.yml
print("🏗️ Creating conda environment from env.yml...")
print("⏳ This will take 5-10 minutes...")
!mamba env create -f env.yml

print("✅ Conda environment created successfully!")

# Verify installation
print("\n🔍 Verifying environment...")
verification_cmd = '''
source /content/miniconda/bin/activate sat_env && python -c "
import torch
import torchvision 
import timm
import numpy as np

print('✅ PyTorch:', torch.__version__)
print('✅ torchvision:', torchvision.__version__)
print('✅ timm:', timm.__version__)
print('✅ numpy:', np.__version__)

# Test critical SatMAE imports
from timm.models.layers import trunc_normal_
from timm.data.mixup import Mixup
from timm.loss import LabelSmoothingCrossEntropy, SoftTargetCrossEntropy
print('✅ SatMAE-critical timm imports successful')

if torch.cuda.is_available():
    print('✅ CUDA:', torch.cuda.get_device_name(0))
    print('✅ GPU Memory: {:.1f} GB'.format(torch.cuda.get_device_properties(0).total_memory / 1e9))
else:
    print('⚠️ CUDA not available')

print('🚀 Environment ready for SatMAE!')
"
'''

result = subprocess.run(verification_cmd, shell=True, capture_output=True, text=True)
print(result.stdout)
if result.stderr:
    print("Warnings:", result.stderr)

print("\n" + "="*50)
print("🎉 CONDA ENVIRONMENT READY!")
print("="*50)

Installing required packages...
✅ timm version: 1.0.17
✅ rasterio version: 1.4.3
✅ gdown installed for Google Drive downloads
All packages installed successfully!


## 3. Mount Google Drive

**Required Google Drive Structure:**
```
MyDrive/
├── data/                           # Unzipped EuroSAT dataset folder
│   ├── eurosat_ms/                 # Multispectral dataset
│   └── eurosat_rgb/                # RGB dataset  
└── checkpoint/
    └── pretrain-vit-large-e199.pth # Pretrained model checkpoint
```

**Setup Steps:**
1. Unzip your EuroSAT dataset and upload the `data/` folder to Google Drive root
2. Upload checkpoint to `MyDrive/checkpoint/pretrain-vit-large-e199.pth`
3. Run the cells below

In [ ]:
# Mount Google Drive and access data directly (no copying needed!)
from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Define data paths - use Google Drive directly
data_root = '/content/drive/MyDrive/data'
eurosat_ms_path = os.path.join(data_root, 'eurosat_ms')
eurosat_rgb_path = os.path.join(data_root, 'eurosat_rgb')

print("🔍 Checking data availability in Google Drive...")

# Verify eurosat_ms exists
if os.path.exists(eurosat_ms_path):
    print(f"✅ Found eurosat_ms at: {eurosat_ms_path}")
    ms_classes = os.listdir(eurosat_ms_path)
    print(f"   📊 Classes: {len(ms_classes)} ({', '.join(ms_classes[:3])}...)")
else:
    print(f"❌ eurosat_ms not found at: {eurosat_ms_path}")

# Verify eurosat_rgb exists  
if os.path.exists(eurosat_rgb_path):
    print(f"✅ Found eurosat_rgb at: {eurosat_rgb_path}")
    rgb_classes = os.listdir(eurosat_rgb_path)
    print(f"   📊 Classes: {len(rgb_classes)} ({', '.join(rgb_classes[:3])}...)")
else:
    print(f"❌ eurosat_rgb not found at: {eurosat_rgb_path}")

# Create data_splits directory in SatMAE folder (for txt files)
splits_dir = 'data_splits'
os.makedirs(splits_dir, exist_ok=True)
print(f"📂 Created directory: {os.path.abspath(splits_dir)}/")

print("\n🚀 Data access configured!")
print("💡 Using Google Drive data directly - no copying needed!")
print(f"📁 MS data: {eurosat_ms_path}")
print(f"📁 RGB data: {eurosat_rgb_path}")
print(f"📁 Splits will be saved to: {os.path.abspath(splits_dir)}/")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
✅ Data copied from /content/drive/MyDrive/data to SatMAE/data

📁 Data structure:
total 16
drwx------  4 root root 4096 Jul 25 13:33 .
drwxr-xr-x  8 root root 4096 Jul 25 13:47 ..
drwx------ 12 root root 4096 Jul 25 13:33 eurosat_ms
drwx------ 12 root root 4096 Jul 25 13:36 eurosat_rgb


In [ ]:
# Load pretrained checkpoint from Google Drive
import os
import shutil

# Define paths (we're already in SatMAE directory after %cd SatMAE)
drive_checkpoint_path = '/content/drive/MyDrive/checkpoint/pretrain-vit-large-e199.pth'
local_checkpoint_dir = 'checkpoints'  # Fixed: removed SatMAE/ prefix
local_checkpoint_path = 'checkpoints/pretrain-vit-large-e199.pth'  # Fixed: removed SatMAE/ prefix

print("🔧 Loading pretrained checkpoint from Google Drive...")
print(f"Source: {drive_checkpoint_path}")
print(f"Target: {local_checkpoint_path}")

# Create checkpoints directory
os.makedirs(local_checkpoint_dir, exist_ok=True)
print(f"📂 Created directory: {os.path.abspath(local_checkpoint_dir)}/")

# Check if checkpoint exists in Google Drive
if os.path.exists(drive_checkpoint_path):
    print(f"✅ Found checkpoint in Google Drive")
    print(f"📊 File size: {os.path.getsize(drive_checkpoint_path) / 1e6:.1f} MB")

    # Copy checkpoint to local directory
    try:
        shutil.copy2(drive_checkpoint_path, local_checkpoint_path)
        print(f"✅ Checkpoint copied successfully!")
        print(f"📁 Available at: {os.path.abspath(local_checkpoint_path)}")

        # Verify the file
        if os.path.exists(local_checkpoint_path):
            print(f"✅ Verification successful - checkpoint ready for training")
        else:
            print(f"❌ Verification failed - file not found at target location")

    except Exception as e:
        print(f"❌ Copy failed: {e}")

else:
    print(f"❌ Checkpoint not found at: {drive_checkpoint_path}")
    print("Please ensure you have uploaded the checkpoint to your Google Drive")
    print("\nTo fix this:")
    print("1. Go to your Google Drive")
    print("2. Create a folder called 'checkpoint' in the root directory")
    print("3. Upload 'pretrain-vit-large-e199.pth' to MyDrive/checkpoint/")
    print("4. Run this cell again")

    print(f"\nExpected Google Drive structure:")
    print(f"  MyDrive/")
    print(f"  ├── data/")
    print(f"  │   ├── eurosat_ms/")
    print(f"  │   └── eurosat_rgb/")
    print(f"  └── checkpoint/")
    print(f"      └── pretrain-vit-large-e199.pth")

🔧 Loading pretrained checkpoint from Google Drive...
Source: /content/drive/MyDrive/checkpoint/pretrain-vit-large-e199.pth
Target: SatMAE/checkpoints/pretrain-vit-large-e199.pth
📂 Created directory: SatMAE/checkpoints/
✅ Found checkpoint in Google Drive
📊 File size: 298.8 MB
✅ Checkpoint copied successfully!
📁 Available at: SatMAE/checkpoints/pretrain-vit-large-e199.pth
✅ Verification successful - checkpoint ready for training


## 4. Data Preprocessing

### 4.1 Generate txt Files and Training Subsets

The text files are used for loading Eurosat Data stored in `SatMAE\data\`. They look like this:

```
<path_to_image> <label>
```
For example:
```
/path/to/image1.tif    0
/path/to/image2.tif    3
...
```

The .txt-files are generated from the script below:

In [ ]:
# Create train/val splits and subsets using Google Drive data directly
import os
from glob import glob
import random

def generate_split_txt(root_folder, out_txt_path, split_ratio=0.8, seed=42):
    """
    Creates train/val .txt files from a root image folder organized by class.
    Supports .tif and .jpg files.
    """
    if not os.path.exists(root_folder):
        print(f"❌ Data folder not found: {root_folder}")
        return
        
    class_names = sorted(os.listdir(root_folder))
    class_to_idx = {cls: idx for idx, cls in enumerate(class_names)}

    all_samples = []
    for cls in class_names:
        tif_paths = glob(os.path.join(root_folder, cls, "*.tif"))
        jpg_paths = glob(os.path.join(root_folder, cls, "*.jpg"))
        image_paths = tif_paths + jpg_paths
        for path in image_paths:
            all_samples.append(f"{path} {class_to_idx[cls]}")

    if not all_samples:
        print(f"⚠️  No image files found in: {root_folder}")
        return

    random.seed(seed)
    random.shuffle(all_samples)
    split_idx = int(len(all_samples) * split_ratio)
    train_samples = all_samples[:split_idx]
    val_samples = all_samples[split_idx:]

    # Save to SatMAE/data_splits directory (maintain expected structure)
    splits_dir = 'data_splits'
    os.makedirs(splits_dir, exist_ok=True)
    train_path = out_txt_path.replace(".txt", "_train.txt")
    val_path = out_txt_path.replace(".txt", "_val.txt")
    
    with open(train_path, "w") as f:
        f.write("\n".join(train_samples))
    with open(val_path, "w") as f:
        f.write("\n".join(val_samples))

    print(f"✅ Created train/val splits for: {root_folder}")
    print(f"   → Train: {len(train_samples)} samples → {train_path}")
    print(f"   → Val:   {len(val_samples)} samples → {val_path}")

# Use Google Drive paths directly
eurosat_ms_path = '/content/drive/MyDrive/data/eurosat_ms'
eurosat_rgb_path = '/content/drive/MyDrive/data/eurosat_rgb'

print("🔄 Generating splits from Google Drive data...")

# Generate splits using Google Drive data directly, save to SatMAE/data_splits/
generate_split_txt(eurosat_ms_path, "data_splits/eurosat_ms.txt")
generate_split_txt(eurosat_rgb_path, "data_splits/eurosat_rgb.txt")

✅ Created train/val splits for: SatMAE/data/eurosat_ms
   → Train: 21600 samples
   → Val:   5400 samples
✅ Created train/val splits for: SatMAE/data/eurosat_rgb
   → Train: 21600 samples
   → Val:   5400 samples



### 3.2 **Create Training Subsets (10%, 25%, 50%, 100%)**

The Goal is to measure how model performance improves as the training data size increases. To ensure fair and meaningful comparisons across runs, the validation set remains fixed.

The following textfiles were generated and include the complete dataset:

```
SatMAE/data_splits/eurosat_ms_train.txt
SatMAE/data_splits/eurosat_rgb_train.txt
```

To subsample:

* Randomly select a percentage of lines from that file
* Save them into new files like:

  ```
  SatMAE/data_splits/eurosat_ms_train_10.txt
  SatMAE/data_splits/eurosat_ms_train_25.txt
  SatMAE/data_splits/eurosat_ms_train_50.txt
  ```

Do this for RGB and MS too:

In [ ]:
def subsample_txt_file(input_path, output_prefix, percentages=[10, 25, 50], seed=42):
    """Subsample training data to create different dataset sizes."""
    if not os.path.exists(input_path):
        print(f"❌ Input file not found: {input_path}")
        return
        
    with open(input_path, 'r') as f:
        lines = f.readlines()

    random.seed(seed)
    random.shuffle(lines)

    for p in percentages:
        count = int(len(lines) * (p / 100))
        subset = lines[:count]
        out_path = f"{output_prefix}_{p}.txt"
        with open(out_path, 'w') as f_out:
            f_out.writelines(subset)
        print(f"Saved {p}% subset to {out_path} ({count} samples)")

print("🔄 Creating training subsets...")

# Use the generated training files in SatMAE/data_splits/
subsample_txt_file("data_splits/eurosat_ms_train.txt", "data_splits/eurosat_ms_train", percentages=[10, 25, 50, 75])
subsample_txt_file("data_splits/eurosat_rgb_train.txt", "data_splits/eurosat_rgb_train", percentages=[10, 25, 50, 75])

print("\n✅ Data preprocessing complete!")
print("\n📁 Generated files:")
!ls -la data_splits/

print("\n💡 All txt files contain Google Drive paths:")
print("   Example paths point to /content/drive/MyDrive/data/eurosat_ms/...")
print("   No local data copying needed! 🚀")

Saved 10% subset to SatMAE/data_splits/eurosat_ms_train_10.txt (2160 samples)
Saved 25% subset to SatMAE/data_splits/eurosat_ms_train_25.txt (5400 samples)
Saved 50% subset to SatMAE/data_splits/eurosat_ms_train_50.txt (10800 samples)
Saved 75% subset to SatMAE/data_splits/eurosat_ms_train_75.txt (16200 samples)
Saved 10% subset to SatMAE/data_splits/eurosat_rgb_train_10.txt (2160 samples)
Saved 25% subset to SatMAE/data_splits/eurosat_rgb_train_25.txt (5400 samples)
Saved 50% subset to SatMAE/data_splits/eurosat_rgb_train_50.txt (10800 samples)
Saved 75% subset to SatMAE/data_splits/eurosat_rgb_train_75.txt (16200 samples)

✅ Data preprocessing complete!

📁 Generated files:
total 6732
drwxr-xr-x 3 root root    4096 Jul 25 14:14 .
drwxr-xr-x 9 root root    4096 Jul 25 14:01 ..
-rw-r--r-- 1 root root  119137 Jul 25 14:15 eurosat_ms_train_10.txt
-rw-r--r-- 1 root root  298594 Jul 25 14:15 eurosat_ms_train_25.txt
-rw-r--r-- 1 root root  595927 Jul 25 14:15 eurosat_ms_train_50.txt
-rw-r--r

## 4. Model Training

In [ ]:
# Verify all required files exist
import os

required_files = [
    'main_finetune.py',  # We're in SatMAE directory
    'data_splits/eurosat_ms_train_10.txt',  # Local txt files
    'data_splits/eurosat_ms_val.txt',
    'checkpoints/pretrain-vit-large-e199.pth'
]

# Also check Google Drive data access
gdrive_paths = [
    '/content/drive/MyDrive/data/eurosat_ms',
    '/content/drive/MyDrive/checkpoint/pretrain-vit-large-e199.pth'
]

print("Checking required files:")
all_good = True

# Check local files
for file in required_files:
    if os.path.exists(file):
        print(f"✅ {file}")
    else:
        print(f"❌ {file} - MISSING")
        all_good = False

# Check Google Drive access
print("\nChecking Google Drive data access:")
for path in gdrive_paths:
    if os.path.exists(path):
        print(f"✅ {path}")
    else:
        print(f"❌ {path} - MISSING")
        all_good = False

# Verify txt files contain valid paths
if os.path.exists('data_splits/eurosat_ms_train_10.txt'):
    with open('data_splits/eurosat_ms_train_10.txt', 'r') as f:
        first_line = f.readline().strip()
        if first_line:
            image_path = first_line.split()[0]
            if os.path.exists(image_path):
                print(f"✅ Sample image accessible: {image_path}")
            else:
                print(f"❌ Sample image not accessible: {image_path}")
                all_good = False

if all_good:
    print("\n🚀 All files ready for training!")
    print("💡 Using Google Drive data directly - fast and efficient!")
else:
    print("\n⚠️ Some files are missing. Please check the previous steps.")

In [ ]:
# Run SatMAE finetuning in conda environment
import subprocess

# Get GPU info in conda environment
gpu_cmd = """
source /content/miniconda/bin/activate sat_env && python -c "
import torch
gpu_memory_gb = torch.cuda.get_device_properties(0).total_memory / 1e9 if torch.cuda.is_available() else 0
print(f'{gpu_memory_gb:.1f}')
"
"""

result = subprocess.run(gpu_cmd, shell=True, capture_output=True, text=True)
gpu_memory_gb = float(result.stdout.strip()) if result.stdout.strip() else 0

# Adjust batch_size based on available GPU memory
if gpu_memory_gb >= 15:  # A100, V100
    batch_size = 16
    accum_iter = 8
elif gpu_memory_gb >= 11:  # T4 or similar
    batch_size = 8
    accum_iter = 16
else:  # Smaller GPUs or CPU
    batch_size = 4
    accum_iter = 32

print(f"🚀 GPU Memory: {gpu_memory_gb:.1f}GB")
print(f"🚀 Using batch_size={batch_size}, accum_iter={accum_iter}")

# Training command using conda environment
training_cmd = f"""source /content/miniconda/bin/activate sat_env && python main_finetune.py \\
  --model_type group_c \\
  --model vit_large_patch16 \\
  --dataset_type euro_sat \\
  --train_path data_splits/eurosat_ms_train_10.txt \\
  --test_path data_splits/eurosat_ms_val.txt \\
  --finetune checkpoints/pretrain-vit-large-e199.pth \\
  --input_size 96 --patch_size 8 \\
  --batch_size {batch_size} --accum_iter {accum_iter} \\
  --epochs 30 --blr 2e-4 \\
  --weight_decay 0.05 \\
  --drop_path 0.2 --reprob 0.25 --mixup 0.8 --cutmix 1.0 \\
  --dropped_bands 0 9 10 \\
  --num_workers 2 \\
  --output_dir results/eurosat_ms_10 \\
  --log_dir results/eurosat_ms_10"""

print("\n🚀 Starting SatMAE training in conda environment...")
print("💡 Using Google Drive data directly (no copying needed!)")
print("✅ Running with exact tested package versions (timm 0.3.2, PyTorch 1.11.0)")
print("⏱️ This will take approximately 30-60 minutes depending on GPU")
print("\nCommand:")
print(training_cmd.replace(" &&", " &&\n "))

# Execute training
result = subprocess.run(training_cmd, shell=True)

if result.returncode == 0:
    print("\n🎉 Training completed successfully!")
else:
    print(f"\n❌ Training failed with return code: {result.returncode}")
    print("Check the output above for error details.")

## 5. Monitor Training

In [ ]:
# Load TensorBoard in Colab
%load_ext tensorboard
%tensorboard --logdir SatMAE/results/eurosat_ms_10

print("TensorBoard is running above!")
print("You can monitor training progress, loss curves, and metrics.")

In [ ]:
# Check training results
import os
import glob

results_dir = "SatMAE/results/eurosat_ms_10"
if os.path.exists(results_dir):
    print("Training results:")
    !ls -la {results_dir}

    # Look for log files
    log_files = glob.glob(f"{results_dir}/*.txt")
    if log_files:
        print(f"\nLatest log file: {log_files[-1]}")
        !tail -20 {log_files[-1]}

    # Look for checkpoints
    checkpoints = glob.glob(f"{results_dir}/*.pth")
    if checkpoints:
        print(f"\nCheckpoints created: {len(checkpoints)}")
        for cp in checkpoints[-3:]:
            print(f"  {cp}")
else:
    print("No results directory found. Training may not have started yet.")

## 6. Download Results

In [ ]:
# Package results for download
import zipfile
import os
from datetime import datetime
import glob

def create_results_archive():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    archive_name = f"satmae_results_{timestamp}.zip"

    with zipfile.ZipFile(archive_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Add results directory from SatMAE
        results_path = 'SatMAE/results'
        if os.path.exists(results_path):
            for root, dirs, files in os.walk(results_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, '.')
                    zipf.write(file_path, arcname)

        # Add training logs from SatMAE directory
        log_files = glob.glob('SatMAE/*.log')
        for log_file in log_files:
            zipf.write(log_file)

    return archive_name

if os.path.exists('SatMAE/results'):
    archive_name = create_results_archive()
    print(f"✅ Results packaged in: {archive_name}")
    print(f"File size: {os.path.getsize(archive_name) / 1e6:.1f} MB")
    print("\nYou can download this file using Colab's file panel on the left.")
else:
    print("No results to package yet.")

## 7. Optional: Cleanup and Additional Experiments

In [ ]:
# Run experiments with different data percentages
experiments = [25, 50, 75]

for pct in experiments:
    print(f"\n=== Running experiment with {pct}% of data ===")

    # Adjust epochs based on data size
    epochs = max(10, 30 - (pct // 25) * 5)  # Fewer epochs for more data

    cmd = f"""
    cd SatMAE && python main_finetune.py \
      --model_type group_c \
      --model vit_large_patch16 \
      --dataset_type euro_sat \
      --train_path data_splits/eurosat_ms_train_{pct}.txt \
      --test_path data_splits/eurosat_ms_val.txt \
      --finetune checkpoints/pretrain-vit-large-e199.pth \
      --input_size 96 --patch_size 8 \
      --batch_size {batch_size} --accum_iter {accum_iter} \
      --epochs {epochs} --blr 2e-4 \
      --weight_decay 0.05 \
      --drop_path 0.2 --reprob 0.25 --mixup 0.8 --cutmix 1.0 \
      --dropped_bands 0 9 10 \
      --num_workers 2 \
      --output_dir results/eurosat_ms_{pct} \
      --log_dir results/eurosat_ms_{pct}
    """

    print(f"Training with {pct}% data for {epochs} epochs...")
    !{cmd}

    print(f"Completed {pct}% experiment")

print("\n🎉 All experiments completed!")

In [ ]:
# Package results for download
import zipfile
import os
from datetime import datetime
import glob

def create_results_archive():
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    archive_name = f"satmae_results_{timestamp}.zip"

    with zipfile.ZipFile(archive_name, 'w', zipfile.ZIP_DEFLATED) as zipf:
        # Add results directory from SatMAE
        results_path = 'SatMAE/results'
        if os.path.exists(results_path):
            for root, dirs, files in os.walk(results_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    arcname = os.path.relpath(file_path, '.')
                    zipf.write(file_path, arcname)

        # Add training logs from SatMAE directory
        log_files = glob.glob('SatMAE/*.log')
        for log_file in log_files:
            zipf.write(log_file)

    return archive_name

if os.path.exists('SatMAE/results'):
    archive_name = create_results_archive()
    print(f"✅ Results packaged in: {archive_name}")
    print(f"File size: {os.path.getsize(archive_name) / 1e6:.1f} MB")
    print("\nYou can download this file using Colab's file panel on the left.")
else:
    print("No results to package yet.")